In [4]:
import numpy as np
import seaborn as sns
import pandas as pd
from scipy import stats
import scipy.cluster.hierarchy as hac
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from tslearn.clustering import TimeSeriesKMeans
from python_utils import mode_clusters, percentage_of_cluster_occurrences, concat_long

ImportError: cannot import name 'mode_clusters' from 'python_utils' (/Users/melvinang/anaconda3/lib/python3.7/site-packages/python_utils/__init__.py)

In [2]:
df = pd.read_csv("HourlyEnergyRows.csv")
df.fillna(0, inplace=True)
df.rename(columns={'Unnamed: 0':'Name'}, inplace=True)
df

,Name,2018-06-01 08:00:00,2018-06-01 09:00:00,2018-06-01 10:00:00,2018-06-01 11:00:00,2018-06-01 12:00:00,2018-06-01 13:00:00,2018-06-01 14:00:00,2018-06-01 15:00:00,2018-06-01 16:00:00,...,2019-06-13 23:00:00,2019-06-14 00:00:00,2019-06-14 01:00:00,2019-06-14 02:00:00,2019-06-14 03:00:00,2019-06-14 04:00:00,2019-06-14 05:00:00,2019-06-14 06:00:00,2019-06-14 07:00:00,2019-06-14 08:00:00
0,KenHourlyEnergy,0.0,0.00,0.22,0.23,0.23,0.23,0.23,0.23,0.220000,...,0.72000,0.72000,0.720000,0.780000,35.52,41.900,0.540,0.55,0.560,0.810
1,SivaraamHourlyEnergy,0.0,0.00,1.28,1.27,1.19,1.19,1.20,1.49,1.580001,...,11.66999,21.95000,5.110000,8.210000,7.73,1.680,1.680,1.76,1.680,1.680
2,SnehaHourlyEnergy,0.0,0.00,1.33,1.33,1.36,1.33,1.32,1.32,1.330000,...,3.17000,1.98000,1.980000,1.970000,1.99,1.990,2.000,2.01,2.030,0.790
3,ChristopherHourlyEnergy,0.0,0.00,2.67,2.66,2.65,2.66,2.66,2.71,2.220000,...,124.99000,121.08000,121.110000,91.500000,116.83,72.120,49.900,49.93,49.910,49.930
4,GiriHourlyEnergy,0.0,0.00,0.55,0.56,0.17,0.94,0.56,0.55,0.560000,...,56.27000,57.82000,55.010000,56.110000,54.98,54.790,54.750,54.82,54.870,54.790
5,AjeetHourlyEnergy,0.0,0.00,0.17,0.16,0.17,0.16,0.16,0.16,0.160000,...,9.04999,14.30000,12.430000,14.570000,12.59,1.820,1.820,1.82,1.830,1.820
6,ChayleHourlyEnergy,0.0,0.00,0.00,4.83,3.07,3.07,3.07,3.07,2.400000,...,112.82000,78.26000,79.480000,55.420000,53.68,53.860,50.930,0.00,103.890,49.260
7,RamitHourlyEnergy,0.0,0.00,1.77,1.75,1.76,1.76,1.76,1.76,1.750000,...,38.84000,39.72000,24.280000,18.640000,8.51,1.630,1.640,0.47,1.640,2.790
8,PrakharHourlyEnergy,0.0,0.00,47.21,47.26,47.07,47.60,47.03,47.09,47.200000,...,159.18000,23.41000,182.610000,99.650000,38.95,113.020,56.460,56.56,56.410,56.430
9,KomangHourlyEnergy,0.0,0.00,0.00,1.37,0.17,2.57,1.37,1.37,1.370001,...,0.79000,3.37001,4.419990,4.720000,0.97,0.810,0.810,0.82,0.830,0.830


In [3]:
#remove non-participants
# FinalMatchingData => Keith, Jiayu, Benjamin, Pandarasamy, and Viveka
non_participants = ['Keith', 'Jiayu', 'Benjamin', 'Pandarasamy', 'Viveka']
non_participants = [name + "HourlyEnergy" for name in non_participants]
df = df[~df['Name'].isin(non_participants)]
df.reset_index(drop=True, inplace=True)
df

,Name,2018-06-01 08:00:00,2018-06-01 09:00:00,2018-06-01 10:00:00,2018-06-01 11:00:00,2018-06-01 12:00:00,2018-06-01 13:00:00,2018-06-01 14:00:00,2018-06-01 15:00:00,2018-06-01 16:00:00,...,2019-06-13 23:00:00,2019-06-14 00:00:00,2019-06-14 01:00:00,2019-06-14 02:00:00,2019-06-14 03:00:00,2019-06-14 04:00:00,2019-06-14 05:00:00,2019-06-14 06:00:00,2019-06-14 07:00:00,2019-06-14 08:00:00
0,KenHourlyEnergy,0.0,0.00,0.22,0.23,0.23,0.23,0.23,0.23,0.220000,...,0.72000,0.72000,0.720000,0.780000,35.52,41.90,0.540,0.55,0.560,0.810
1,SivaraamHourlyEnergy,0.0,0.00,1.28,1.27,1.19,1.19,1.20,1.49,1.580001,...,11.66999,21.95000,5.110000,8.210000,7.73,1.68,1.680,1.76,1.680,1.680
2,SnehaHourlyEnergy,0.0,0.00,1.33,1.33,1.36,1.33,1.32,1.32,1.330000,...,3.17000,1.98000,1.980000,1.970000,1.99,1.99,2.000,2.01,2.030,0.790
3,ChristopherHourlyEnergy,0.0,0.00,2.67,2.66,2.65,2.66,2.66,2.71,2.220000,...,124.99000,121.08000,121.110000,91.500000,116.83,72.12,49.900,49.93,49.910,49.930
4,GiriHourlyEnergy,0.0,0.00,0.55,0.56,0.17,0.94,0.56,0.55,0.560000,...,56.27000,57.82000,55.010000,56.110000,54.98,54.79,54.750,54.82,54.870,54.790
5,AjeetHourlyEnergy,0.0,0.00,0.17,0.16,0.17,0.16,0.16,0.16,0.160000,...,9.04999,14.30000,12.430000,14.570000,12.59,1.82,1.820,1.82,1.830,1.820
6,ChayleHourlyEnergy,0.0,0.00,0.00,4.83,3.07,3.07,3.07,3.07,2.400000,...,112.82000,78.26000,79.480000,55.420000,53.68,53.86,50.930,0.00,103.890,49.260
7,RamitHourlyEnergy,0.0,0.00,1.77,1.75,1.76,1.76,1.76,1.76,1.750000,...,38.84000,39.72000,24.280000,18.640000,8.51,1.63,1.640,0.47,1.640,2.790
8,PrakharHourlyEnergy,0.0,0.00,47.21,47.26,47.07,47.60,47.03,47.09,47.200000,...,159.18000,23.41000,182.610000,99.650000,38.95,113.02,56.460,56.56,56.410,56.430
9,KomangHourlyEnergy,0.0,0.00,0.00,1.37,0.17,2.57,1.37,1.37,1.370001,...,0.79000,3.37001,4.419990,4.720000,0.97,0.81,0.810,0.82,0.830,0.830


In [4]:
# split into time before, during and after the game
start_date = '2018-08-02'
end_date = '2018-10-10'
before = [col for col in df.columns if col <= start_date]
during = [col for col in df.columns if col > start_date and col <= end_date]
after = [col for col in df.columns if col > end_date]

In [5]:
df_before = df.loc[:, before]
df_before

,2018-06-01 08:00:00,2018-06-01 09:00:00,2018-06-01 10:00:00,2018-06-01 11:00:00,2018-06-01 12:00:00,2018-06-01 13:00:00,2018-06-01 14:00:00,2018-06-01 15:00:00,2018-06-01 16:00:00,2018-06-01 17:00:00,...,2018-08-01 14:00:00,2018-08-01 15:00:00,2018-08-01 16:00:00,2018-08-01 17:00:00,2018-08-01 18:00:00,2018-08-01 19:00:00,2018-08-01 20:00:00,2018-08-01 21:00:00,2018-08-01 22:00:00,2018-08-01 23:00:00
0,0.0,0.00,0.22,0.23,0.23,0.23,0.23,0.23,0.220000,0.220000,...,1.860000,1.850000,1.860000,1.8300,1.850,54.07,55.56,89.78000,13.60000,32.460
1,0.0,0.00,1.28,1.27,1.19,1.19,1.20,1.49,1.580001,1.269999,...,0.350000,0.350000,0.640000,0.3500,9.130,0.68,0.37,0.37000,0.37000,0.370
2,0.0,0.00,1.33,1.33,1.36,1.33,1.32,1.32,1.330000,1.220000,...,0.790000,0.790000,0.790000,0.7900,0.780,29.53,80.53,102.96000,93.26000,81.450
3,0.0,0.00,2.67,2.66,2.65,2.66,2.66,2.71,2.220000,3.120000,...,2.159999,2.160000,2.150000,2.1400,3.420,74.03,143.31,117.86000,109.83000,113.520
4,0.0,0.00,0.55,0.56,0.17,0.94,0.56,0.55,0.560000,0.540000,...,0.050000,0.000000,0.050000,0.0000,0.430,26.70,39.46,29.42000,24.86000,15.340
5,0.0,0.00,0.17,0.16,0.17,0.16,0.16,0.16,0.160000,0.160000,...,0.000000,0.000000,0.000000,0.0000,11.610,53.82,19.08,14.46000,14.29000,13.890
6,0.0,0.00,0.00,4.83,3.07,3.07,3.07,3.07,2.400000,3.720000,...,47.860000,47.820000,48.120000,47.8100,47.810,47.80,126.23,124.74000,49.62000,111.780
7,0.0,0.00,1.77,1.75,1.76,1.76,1.76,1.76,1.750000,1.760000,...,1.640000,1.650000,1.630000,1.6300,9.530,62.64,56.30,54.77000,57.76000,56.880
8,0.0,0.00,47.21,47.26,47.07,47.60,47.03,47.09,47.200000,46.970000,...,46.240000,46.160000,46.140000,46.3400,46.250,46.18,45.99,46.87010,78.81990,83.770
9,0.0,0.00,0.00,1.37,0.17,2.57,1.37,1.37,1.370001,1.379999,...,0.000000,0.000000,0.000000,0.0700,4.260,48.78,0.15,25.50000,1.76000,0.000


In [6]:
df_during = df.loc[:, during]
df_during

,2018-08-02 00:00:00,2018-08-02 01:00:00,2018-08-02 02:00:00,2018-08-02 03:00:00,2018-08-02 04:00:00,2018-08-02 05:00:00,2018-08-02 06:00:00,2018-08-02 07:00:00,2018-08-02 08:00:00,2018-08-02 09:00:00,...,2018-10-09 14:00:00,2018-10-09 15:00:00,2018-10-09 16:00:00,2018-10-09 17:00:00,2018-10-09 18:00:00,2018-10-09 19:00:00,2018-10-09 20:00:00,2018-10-09 21:00:00,2018-10-09 22:00:00,2018-10-09 23:00:00
0,0.92000,0.92000,0.93,0.00,94.410,0.00,0.00,0.00,0.00000,0.0,...,0.28,0.28,0.54,0.270000,0.820000,69.170000,73.06000,69.45000,30.50000,84.00
1,0.37000,0.37000,0.37,0.36,0.350,0.66,0.35,0.65,0.66000,0.0,...,0.40,0.48,0.48,1.070000,34.330000,34.760000,7.52001,34.83999,34.66000,16.72
2,49.39000,119.69000,80.17,27.04,0.830,0.79,0.79,0.79,0.79000,0.0,...,0.79,0.79,0.78,0.770000,0.770000,17.760000,82.74000,98.73000,83.63000,84.01
3,80.49000,78.20000,79.51,77.48,49.560,2.58,2.58,2.16,2.51000,0.0,...,2.16,2.15,2.14,2.130000,3.120000,2.640000,108.40000,87.23000,143.73000,115.42
4,8.94000,8.94000,8.93,9.57,0.150,0.05,0.00,0.00,0.00000,0.0,...,0.00,0.00,0.00,0.000000,0.080000,1.210000,22.35000,27.82000,24.97000,24.30
5,16.38000,17.07000,5.86,0.29,0.330,0.00,0.00,0.07,0.00000,0.0,...,0.00,0.07,0.07,0.000000,0.000000,73.965010,0.29000,0.29000,0.35999,42.21
6,146.90000,64.16000,137.84,50.29,107.000,133.05,5.39,5.69,5.38000,0.0,...,67.02,65.83,54.44,54.370000,54.360000,114.520000,126.90000,105.01000,133.68000,111.87
7,59.68000,66.22000,59.63,62.92,67.290,14.50,16.42,3.38,1.74000,0.0,...,0.98,1.96,0.05,0.980001,6.229999,25.390000,34.72001,43.45999,21.61000,59.03
8,64.05000,46.71000,73.87,48.45,0.000,94.27,47.08,47.71,47.06000,0.0,...,78.43,54.41,48.30,48.640000,48.190000,57.480000,79.71000,112.75000,64.53000,66.79
9,2.93000,1.11000,0.85,1.35,0.070,0.04,0.00,0.00,0.00000,0.0,...,0.00,0.00,0.00,0.070000,0.000000,0.000000,0.06000,0.00000,0.00000,24.28


In [7]:
df_after = df.loc[:, after].drop(columns = ['Name'])
df_after

,2018-10-10 00:00:00,2018-10-10 01:00:00,2018-10-10 02:00:00,2018-10-10 03:00:00,2018-10-10 04:00:00,2018-10-10 05:00:00,2018-10-10 06:00:00,2018-10-10 07:00:00,2018-10-10 08:00:00,2018-10-10 09:00:00,...,2019-06-13 23:00:00,2019-06-14 00:00:00,2019-06-14 01:00:00,2019-06-14 02:00:00,2019-06-14 03:00:00,2019-06-14 04:00:00,2019-06-14 05:00:00,2019-06-14 06:00:00,2019-06-14 07:00:00,2019-06-14 08:00:00
0,74.93000,74.83,65.20,64.70000,29.110,8.64,0.28,0.28,0.28,0.0,...,0.72000,0.72000,0.720000,0.780000,35.52,41.90,0.540,0.55,0.560,0.810
1,12.82000,32.43,23.75,6.26000,0.400,0.40,0.40,0.40,0.40,0.0,...,11.66999,21.95000,5.110000,8.210000,7.73,1.68,1.680,1.76,1.680,1.680
2,85.96000,84.38,89.44,70.94000,1.030,0.80,0.81,0.81,0.89,0.0,...,3.17000,1.98000,1.980000,1.970000,1.99,1.99,2.000,2.01,2.030,0.790
3,74.97000,158.40,108.03,111.95000,73.220,3.67,2.71,2.65,2.28,0.0,...,124.99000,121.08000,121.110000,91.500000,116.83,72.12,49.900,49.93,49.910,49.930
4,13.33000,9.18,9.20,14.75000,0.080,0.00,0.00,0.05,0.00,0.0,...,56.27000,57.82000,55.010000,56.110000,54.98,54.79,54.750,54.82,54.870,54.790
5,26.88000,11.47,9.37,0.00000,5.045,0.00,0.00,0.00,0.24,0.0,...,9.04999,14.30000,12.430000,14.570000,12.59,1.82,1.820,1.82,1.830,1.820
6,110.01000,113.40,141.41,120.63000,100.870,77.69,70.61,53.93,53.92,0.0,...,112.82000,78.26000,79.480000,55.420000,53.68,53.86,50.930,0.00,103.890,49.260
7,29.25000,56.37,40.68,2.46000,4.570,1.02,0.98,0.05,1.96,0.0,...,38.84000,39.72000,24.280000,18.640000,8.51,1.63,1.640,0.47,1.640,2.790
8,107.08000,85.50,84.31,76.33000,86.140,72.98,58.34,47.94,48.20,0.0,...,159.18000,23.41000,182.610000,99.650000,38.95,113.02,56.460,56.56,56.410,56.430
9,43.57000,15.97,5.84,0.00000,0.000,0.00,0.07,0.00,0.05,0.0,...,0.79000,3.37001,4.419990,4.720000,0.97,0.81,0.810,0.82,0.830,0.830


In [8]:
def split_in_weeks(df1): 
    # every week = every 7*24 timeframes
    N = 7*24
    weeks = np.split(df1, np.arange(N, len(df1.columns), N), axis=1)
    return weeks

In [9]:
def normalize_series(dfs): 
    def normalize(df1): 
        normalized = df1.copy().div(df1.sum(axis=1), axis=0)
        normalized.fillna(0, inplace=True)
        return normalized
    
    normalized_series = list(map(normalize, dfs))
    return normalized_series

In [10]:
def runKMeans(df1, k=3, metric="dtw"): 
    results = pd.DataFrame()
    results = pd.concat([results, df['Name']], axis=1)
    
    train = concat_long(df1).fillna(0)
    kmeans = TimeSeriesKMeans(n_clusters=k, metric=metric, random_state=0, n_init=1).fit(train)
    sum_of_squared_distances = kmeans.inertia_
    for period in df1: 
        try: 
            predictions = pd.DataFrame(kmeans.predict(period), columns=["{} - {}".format(period.columns[0],period.columns[-1])])
            results = pd.concat([results, predictions], axis=1)
        except ValueError as err: 
            print("Could not predict cluster for period. {}".format(err))
            
    return results, sum_of_squared_distances, kmeans

### Predicting before

In [11]:
weeks_before = normalize_series(split_in_weeks(df_before))
print(len(weeks_before))
weeks_before

9


       0.000471   
 15             0.003926             0.003995             0.004011   
 16             0.003643             0.004621             0.006750   
 17             0.000660             0.000297             0.000000   
 18             0.000547             0.000547             0.000538   
 19             0.001011             0.001519             0.000581   
 20             0.000988             0.001263             0.000712   
 21             0.006347             0.006342             0.006349   
 22             0.007323             0.007567             0.007096   
 
     2018-07-20 14:00:00  2018-07-20 15:00:00  2018-07-20 16:00:00  \
 0              0.000495             0.000495             0.000499   
 1              0.001057             0.002527             0.004569   
 2              0.000444             0.000568             0.000506   
 3              0.000484             0.000413             0.000342   
 4              0.006013             0.005741             0.006052   

In [12]:
results_before, _, _ = runKMeans(weeks_before, k=3)

c:\Users\U566429\Desktop\Studium Nordakadmie\5. Semester\IEOR 242 - Applications in Data Analysis\Final Project\SocialGameDataHourly\utils.py:113: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  result = pd.concat([result, period], axis=0, ignore_index=True)
C:\Users\U566429\AppData\Local\Continuum\anaconda3\lib\site-packages\tslearn\utils.py:63: UserWarning: 2-Dimensional data passed. Assuming these are 23 1-dimensional timeseries
  '{} 1-dimensional timeseries'.format(X.shape[0]))
C:\Users\U566429\AppData\Local\Continuum\anaconda3\lib\site-packages\tslearn\utils.py:63: UserWarning: 2-Dimensional data passed. Assuming these are 23 1-dimensional timeseries
  '{} 1-dimensional timeseries'.format(X.shape[0]))
C:\Users\U566429\AppData\Local\Continuum\anaconda3\lib\site-packages\

In [13]:
results_before

,Name,2018-06-01 08:00:00 - 2018-06-08 07:00:00,2018-06-08 08:00:00 - 2018-06-15 07:00:00,2018-06-15 08:00:00 - 2018-06-22 07:00:00,2018-06-22 08:00:00 - 2018-06-29 07:00:00,2018-06-29 08:00:00 - 2018-07-06 07:00:00,2018-07-06 08:00:00 - 2018-07-13 07:00:00,2018-07-13 08:00:00 - 2018-07-20 07:00:00,2018-07-20 08:00:00 - 2018-07-27 07:00:00
0,KenHourlyEnergy,2,0,0,2,1,1,1,1
1,SivaraamHourlyEnergy,0,2,1,2,2,2,1,1
2,SnehaHourlyEnergy,1,1,1,1,1,2,1,1
3,ChristopherHourlyEnergy,2,1,2,2,2,1,2,2
4,GiriHourlyEnergy,0,1,1,0,2,0,1,1
5,AjeetHourlyEnergy,0,0,0,0,0,1,2,1
6,ChayleHourlyEnergy,2,2,2,2,2,2,2,2
7,RamitHourlyEnergy,2,1,1,2,1,2,2,2
8,PrakharHourlyEnergy,2,2,2,2,2,2,2,2
9,KomangHourlyEnergy,0,1,1,1,1,0,1,0


In [14]:
mode_clusters(results_before)

,Name,Mode cluster
0,KenHourlyEnergy,1
1,SivaraamHourlyEnergy,2
2,SnehaHourlyEnergy,1
3,ChristopherHourlyEnergy,2
4,GiriHourlyEnergy,1
5,AjeetHourlyEnergy,0
6,ChayleHourlyEnergy,2
7,RamitHourlyEnergy,2
8,PrakharHourlyEnergy,2
9,KomangHourlyEnergy,1


In [15]:
# Calculate percentage of cluster occurrences per person
percentage_of_cluster_occurrences(results_before)

,Name,OccurenceOfCluster0,OccurenceOfCluster1
0,KenHourlyEnergy,0.250,0.500
1,SivaraamHourlyEnergy,0.125,0.375
2,SnehaHourlyEnergy,0.000,0.875
3,ChristopherHourlyEnergy,0.000,0.250
4,GiriHourlyEnergy,0.375,0.500
5,AjeetHourlyEnergy,0.625,0.250
6,ChayleHourlyEnergy,0.000,0.000
7,RamitHourlyEnergy,0.000,0.375
8,PrakharHourlyEnergy,0.000,0.000
9,KomangHourlyEnergy,0.375,0.625


## Predicting during

In [16]:
weeks_during = normalize_series(split_in_weeks(df_during))
print(len(weeks_during))
weeks_during

10


       0.000000   
 15             0.003792             0.003592             0.003590   
 16             0.087895             0.009329             0.000000   
 17             0.000000             0.000544             0.000000   
 18             0.005481             0.012376             0.009296   
 19             0.014211             0.000506             0.000000   
 20             0.009298             0.000918             0.000356   
 21             0.005990             0.007365             0.000000   
 22             0.006731             0.010386             0.008439   
 
     2018-09-27 06:00:00  2018-09-27 07:00:00  2018-09-27 08:00:00  \
 0              0.000141             0.000112             0.000116   
 1              0.001003             0.001003             0.001003   
 2              0.023228             0.023258             0.023233   
 3              0.000799             0.000780             0.000669   
 4              0.000000             0.000138             0.000000   

In [17]:
results_during, _, _ = runKMeans(weeks_during, k=3)

c:\Users\U566429\Desktop\Studium Nordakadmie\5. Semester\IEOR 242 - Applications in Data Analysis\Final Project\SocialGameDataHourly\utils.py:113: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  result = pd.concat([result, period], axis=0, ignore_index=True)
C:\Users\U566429\AppData\Local\Continuum\anaconda3\lib\site-packages\tslearn\utils.py:63: UserWarning: 2-Dimensional data passed. Assuming these are 23 1-dimensional timeseries
  '{} 1-dimensional timeseries'.format(X.shape[0]))
C:\Users\U566429\AppData\Local\Continuum\anaconda3\lib\site-packages\tslearn\utils.py:63: UserWarning: 2-Dimensional data passed. Assuming these are 23 1-dimensional timeseries
  '{} 1-dimensional timeseries'.format(X.shape[0]))
C:\Users\U566429\AppData\Local\Continuum\anaconda3\lib\site-packages\

In [18]:
results_during

,Name,2018-08-02 00:00:00 - 2018-08-08 23:00:00,2018-08-09 00:00:00 - 2018-08-15 23:00:00,2018-08-16 00:00:00 - 2018-08-22 23:00:00,2018-08-23 00:00:00 - 2018-08-29 23:00:00,2018-08-30 00:00:00 - 2018-09-05 23:00:00,2018-09-06 00:00:00 - 2018-09-12 23:00:00,2018-09-13 00:00:00 - 2018-09-19 23:00:00,2018-09-20 00:00:00 - 2018-09-26 23:00:00,2018-09-27 00:00:00 - 2018-10-03 23:00:00
0,KenHourlyEnergy,1,1,1,2,1,1,1,2,2
1,SivaraamHourlyEnergy,1,1,2,2,2,2,2,2,1
2,SnehaHourlyEnergy,1,2,2,2,2,1,2,1,2
3,ChristopherHourlyEnergy,2,2,2,2,2,2,2,2,2
4,GiriHourlyEnergy,1,2,1,0,0,1,0,1,1
5,AjeetHourlyEnergy,1,1,1,0,1,1,0,0,0
6,ChayleHourlyEnergy,2,2,2,2,2,2,2,2,2
7,RamitHourlyEnergy,2,2,2,2,1,2,2,2,2
8,PrakharHourlyEnergy,2,2,2,1,1,2,2,2,2
9,KomangHourlyEnergy,0,0,0,0,0,0,0,0,0


In [19]:
mode_clusters(results_during)

,Name,Mode cluster
0,KenHourlyEnergy,1
1,SivaraamHourlyEnergy,2
2,SnehaHourlyEnergy,2
3,ChristopherHourlyEnergy,2
4,GiriHourlyEnergy,1
5,AjeetHourlyEnergy,1
6,ChayleHourlyEnergy,2
7,RamitHourlyEnergy,2
8,PrakharHourlyEnergy,2
9,KomangHourlyEnergy,0


In [20]:
# Calculate percentage of cluster occurrences per person
percentage_of_cluster_occurrences(results_during)

,Name,OccurenceOfCluster0,OccurenceOfCluster1
0,KenHourlyEnergy,0.0000,0.6667
1,SivaraamHourlyEnergy,0.0000,0.3333
2,SnehaHourlyEnergy,0.0000,0.3333
3,ChristopherHourlyEnergy,0.0000,0.0000
4,GiriHourlyEnergy,0.3333,0.5556
5,AjeetHourlyEnergy,0.4444,0.5556
6,ChayleHourlyEnergy,0.0000,0.0000
7,RamitHourlyEnergy,0.0000,0.1111
8,PrakharHourlyEnergy,0.0000,0.2222
9,KomangHourlyEnergy,1.0000,0.0000


## Predicting after

In [21]:
weeks_after = normalize_series(split_in_weeks(df_after))
print(len(weeks_after))
weeks_after

36


        0.002860   
 15             0.001899             0.002974             0.002279   
 16             0.006879             0.006126             0.006140   
 17             0.001148             0.001148             0.001138   
 18             0.000525             0.000214             0.000335   
 19             0.001530             0.002332             0.000753   
 20             0.000757             0.001074             0.001406   
 21             0.011861             0.003681             0.011247   
 22             0.000145             0.000315             0.000260   
 
     2019-06-05 06:00:00  2019-06-05 07:00:00  2019-06-05 08:00:00  \
 0              0.002779             0.002779             0.002728   
 1              0.001105             0.001105             0.001441   
 2              0.003840             0.003840             0.003821   
 3              0.000000             0.000000             0.000000   
 4              0.007223             0.004042             0.007219  

In [22]:
results_after, _, _ = runKMeans(weeks_after, k=3)

c:\Users\U566429\Desktop\Studium Nordakadmie\5. Semester\IEOR 242 - Applications in Data Analysis\Final Project\SocialGameDataHourly\utils.py:113: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  result = pd.concat([result, period], axis=0, ignore_index=True)


In [0]:
results_after

In [0]:
mode_clusters(results_after)

In [0]:
# Calculate percentage of cluster occurrences per person
percentage_of_cluster_occurrences(results_after)